In [1]:
import numpy as np
from controller_utils import Robot
import rospy
import tools.rotations as rot
%load_ext autoreload
%autoreload 2


In [2]:
optitrack_frame_names=['iiwa_base7', 'iiwa_ee_m']
r = Robot(optitrack_frame_names=optitrack_frame_names, calibration=True)

Kinematic model has been loaded from kinematics/q2pose_left.txt


In [7]:
r.x_obj

{'iiwa_base7': array([ 3.10883881e-02,  6.85629816e-04,  7.64936272e-02,  9.98938905e-01,
         4.23531580e-03, -8.91536691e-03, -4.49849073e-02]),
 'iiwa_ee_m': array([ 0.55901375, -0.16058507,  0.85362214,  0.49812195, -0.78473541,
        -0.36825858, -0.02122434])}

In [3]:
r.iiwa_go_home()

Linear interpolation by 2819 joints
Trajectory has been executed.


In [5]:
r.x_obj

{'iiwa_ee_m': array([-6.34064198e-01, -2.31095719e+00,  1.78158724e+00, -9.37378943e-01,
         1.50641182e-03,  1.93703905e-01,  2.89477587e-01]),
 'iiwa_base7': array([-1.10647011e+00, -1.99815965e+00,  8.80739570e-01, -1.00000000e+00,
         3.25338915e-05, -1.65663660e-05,  1.13584101e-05])}

In [3]:
q_list = []
# for i in range(q_list.shape[0]):
#     print(i)
#     q_list.append(r.q)

# np.save('iiwa_calibration_joints', np.vstack(q_list))
q_list = np.load('iiwa_calibration_joints.npy')
q_list.shape

(20, 7)

In [19]:
nums = 40
# nums = q_list.shape[0]
lb=np.array([0.5, -0.2, -1.5, -1.6, -0.2, -0.3, -0.4])
ub = np.array([0.7, 0.3, -1,  -1.2, 1.5, 0.9, 1.5])
q_rand = np.random.uniform(lb, ub , size=(nums, 7))
q_rand.shape

(40, 7)

In [20]:
pose_list = []
for i in range(nums):
    print(i)
    r.move_to_joints(q_rand[i,:], vel=[0.4, 1])
    rospy.sleep(2)
    #  todo, check if the optitrack frame is visible or not
    pose_list.append([r.x_obj, r.x]) # todo, check if the pose is valid or has been updated, otherwise might some markers are not seen.

0
Linear interpolation by 115 joints
Trajectory has been executed.
1
Linear interpolation by 471 joints
Trajectory has been executed.
2
Linear interpolation by 512 joints
Trajectory has been executed.
3
Linear interpolation by 237 joints
Trajectory has been executed.
4
Linear interpolation by 207 joints
Trajectory has been executed.
5
Linear interpolation by 365 joints
Trajectory has been executed.
6
Linear interpolation by 541 joints
Trajectory has been executed.
7
Linear interpolation by 180 joints
Trajectory has been executed.
8
Linear interpolation by 195 joints
Trajectory has been executed.
9
Linear interpolation by 456 joints
Trajectory has been executed.
10
Linear interpolation by 297 joints
Trajectory has been executed.
11
Linear interpolation by 279 joints
Trajectory has been executed.
12
Linear interpolation by 448 joints
Trajectory has been executed.
13
Linear interpolation by 334 joints
Trajectory has been executed.
14
Linear interpolation by 378 joints
Trajectory has been 

In [21]:
np.save('iiwa_calibration_joints', q_rand)


In [22]:
len(pose_list)

40

In [8]:
r.x_obj

{'iiwa_base7': array([-1.10647225e+00, -1.99813306e+00,  8.80863249e-01, -7.07084000e-01,
         7.07126617e-01,  9.83484206e-04, -1.80687977e-03]),
 'iiwa_ee_m': array([-0.4882383 , -2.13998008,  1.50013793, -0.96507037,  0.16858596,
         0.00539349,  0.20047165])}

In [23]:
for i in pose_list:
    print(i[1])

[ 0.51256919 -0.16702264  0.72031084  0.317049    0.64516174  0.43654598
  0.54099341]
[ 0.37676405 -0.26600089  0.92021996  0.75351561  0.36912573  0.45838356
  0.29299305]
[ 0.40896136 -0.33583411  0.78954401  0.6161845   0.65274991  0.09122782
  0.43117479]
[ 0.52894681 -0.16387901  0.7007101   0.60901963  0.49586183  0.56870872
  0.24451284]
[ 0.31968912 -0.41112991  0.74983738  0.68331933  0.57128059  0.44242841
  0.10473914]
[ 0.36865955 -0.31430926  0.73877463  0.51757427  0.429394    0.67864095
  0.2952696 ]
[ 0.47833395 -0.25519786  0.69951583  0.53499461  0.67375447  0.35082793
  0.3698046 ]
[ 0.39789062 -0.30032343  0.90532752  0.74121189  0.51639783  0.14983983
  0.40185352]
[ 0.52585003 -0.15797324  0.83782594  0.77148977  0.40106462  0.43691562
  0.23033769]
[ 0.49610587 -0.18433537  0.70145683  0.44456509  0.45432076  0.77197669
  0.00255372]
[ 0.37044619 -0.33910138  0.86104615  0.72774126  0.4699507   0.47954557
  0.1399108 ]
[ 0.32098018 -0.37509236  0.91082988  0.802

In [26]:
#Solve the AX=XB problem.
from Pose_Estimation_Class import EKF, Batch_Processing, IEKF, UKF

num=20
A = np.zeros([4,4,int(nums/2)])
B = np.zeros([4,4,int(nums/2)])
for i in range(0, nums, 2):
    T_m12w_1 = rot.pose2T(pose_list[i][0][optitrack_frame_names[0]] )
    T_m22w_1 = rot.pose2T(pose_list[i][0][optitrack_frame_names[1]] )
    T_base2fk_1 = np.linalg.inv(rot.pose2T(pose_list[i][1] ))

    T_m12w_2 = rot.pose2T(pose_list[i+1][0][optitrack_frame_names[0]] )
    T_m22w_2 = rot.pose2T(pose_list[i+1][0][optitrack_frame_names[1]] )
    T_base2fk_2 = np.linalg.inv(rot.pose2T(pose_list[i+1][1] ))


    A[:,:,int(i/2)] = np.linalg.inv(T_m12w_2) @ T_m22w_2 @ np.linalg.inv(T_m22w_1) @ T_m12w_1
    B[:,:,int(i/2)] = np.linalg.inv(T_base2fk_2) @ T_base2fk_1

#Solve the AX=XB problem.
ekf=EKF()
for i in range(len(A[1,1,:])):
    AA=A[:,:,i]
    BB= A[:,:,i]
    ekf.Update(AA,BB)


iekf=IEKF()
for i in range(len(A[1,1,:])):
    AA=A[:,:,i]
    BB=B[:,:,i]
    iekf.Update(AA,BB)


ukf=UKF()
for i in range(len(A[1,1,:])):
    AA=A[:,:,i]
    BB=B[:,:,i]
    ukf.Update(AA,BB)

#Solve the AX=YB problem.
A = np.zeros([4,4,nums])
B = np.zeros([4,4,nums])
for i in range(0, nums):
    T_m12w_1 = rot.pose2T(pose_list[i][0][optitrack_frame_names[0]] )
    T_m22w_1 = rot.pose2T(pose_list[i][0][optitrack_frame_names[1]] )
    T_base2fk_1 = np.linalg.inv(rot.pose2T(pose_list[i][1] ))
    A[:,:,i] = np.linalg.inv(T_m22w_1) @ T_m12w_1
    B[:,:,i] = T_base2fk_1
X_est,Y_est,Y_est_check,ErrorStats=Batch_Processing.pose_estimation(A, B)
x_est = rot.T2pose(X_est) # [x,y,z, qw, qx, qy, qz]

In [27]:
print('position:\n', x_est[:3], '\n', ekf.x[3:], '\n', iekf.x[3:], '\n', ukf.x[3:], '\n' )
print('quaternion:\n', x_est[3:], '\n', rot.axisangle2quat(ekf.x[:3]),'\n',  rot.axisangle2quat(iekf.x[:3]), '\n', rot.axisangle2quat(ukf.x[:3]), '\n')
print('quat error in degree:\n',rot.ori_dis(x_est[3:], rot.axisangle2quat(ekf.x[:3]))*180/np.pi,
        rot.ori_dis(x_est[3:], rot.axisangle2quat(iekf.x[:3]))*180/np.pi,
        rot.ori_dis(x_est[3:], rot.axisangle2quat(ukf.x[:3]))*180/np.pi)

position:
 [-0.03222253 -0.00417823 -0.0759096 ] 
 [-0.04337558 -0.02384747 -0.00014871] 
 [-0.06968535 -0.02570436 -0.06859286] 
 [-0.00113127 -0.08720326  0.19980981] 

quaternion:
 [ 0.99894057 -0.00430084  0.00891559  0.04494156] 
 [ 0.99931547 -0.01620237  0.03270785  0.00602181] 
 [ 0.99527224 -0.01553598  0.07336026  0.06172574] 
 [ 0.9950113  -0.04771517  0.07369961  0.04737235] 

quat error in degree:
 5.402863897029121 7.751891286241216 8.95445395841561


In [29]:
np.savetxt('description/config/iwwa_link_0_2_iiwabase7_calibration.txt', x_est)